In [ ]:
import io
import requests
import json
import urllib.request
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import traceback

In [ ]:
def get_local_img(path):
  with open(path, 'rb') as image_file:
      return image_file.read()

In [ ]:
def getFaces(image_path): #Define uma função "getFaces" que recebe uma url de imagem como argumento
  image_data = get_local_img(image_path)
  face_api_url = 'https://brazilsouth.api.cognitive.microsoft.com/face/v1.0/detect' #Define a url da API de detecção facial da Microsoft
  subscription_key = '' #Define a chave de assinatura para acessar a API
  headers = { 'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key': subscription_key }    #Define os cabeçalhos para a solicitação à API
  params = { #Define os parâmetros para a detecção facial na imagem, incluindo a análise de emoções
      'returnFaceId': 'true',
      'returnFaceLandmarks': 'false',
      'returnFaceAttributes': 'emotion',
  }
  response = requests.post(face_api_url, params=params, headers=headers, data=image_data) #Faz uma solicitação HTTP POST para a API com a imagem e os parâmetros definidos acima
  faces = response.json() #Converte a resposta da API para um objeto JSON
  return faces #Retorna as informações das faces detectadas na imagem

def getEmotion(emotions): #Define uma função "getEmotion" que recebe um dicionário com as emoções de uma face
  values = list( emotions.values() ) #Converte os valores das emoções em uma lista
  index = np.argmax( np.array(values) ) #Obtém o índice da emoção com maior valor na lista
  return list(emotions.keys())[index] #Retorna a chave correspondente à emoção com maior valor

def getEmotionColor(emotion): #Define uma função "getEmotionColor" que recebe uma emoção e retorna sua cor em RGB
  #   descricao           rgb              traducao     cor
  colors = { #Define um dicionário com as cores correspondentes a cada emoção
      'anger':     [ 213, 0, 0 ],         # raiva    | vermelho
      'contempt':  [ 255, 109, 0 ],       # desprezo | laranja escuro
      'disgust':   [ 98, 0, 234 ],        # nojo     | roxo
      'fear':      [ 0, 77, 64 ],         # medo     | preto
      'happiness': [ 255, 234, 0 ],       # alegria  | amarelo
      'neutral':   [ 255, 255, 255 ],     # neutro   | branco
      'sadness':   [ 41, 98, 255 ],       # triste   | azul
      'surprise':  [ 0, 229, 255  ],      # surpreso | azul claro
  }
  return np.array( colors[emotion], dtype=np.uint8 ) #Retorna a cor correspondente à emoção recebida

# para calcular a distância entre o centro do retângulo que circunda o rosto e cada pixel dentro desse retângulo.
def euclidianDistance(p1, p2): #Define uma função "euclidianDistance" que calcula a distância euclidiana entre dois pontos
  return math.sqrt( (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 ) #Calcula a distância euclidiana entre os pontos

def markEmotionOnFace(face, mat):
  rect = face['faceRectangle'] #Essas linhas recuperam as coordenadas do retângulo que circunda o rosto (rect) a partir do dicionário face
  col1, col2 = rect['left'], rect['left'] + rect['width'] #variáveis col1, col2, row1 e row2 são definidas a partir das coordenadas left, width, top e height do retângulo, respectivamente
  row1, row2 = rect['top'], rect['top'] + rect['height']

  emotions = face['faceAttributes']['emotion'] #emoções detectadas no rosto (emotions) a partir do dicionário face, passam essas emoções para a função getEmotion para obter a emoção dominante (emotion)
  #Em seguida, chamam a função getEmotionColor para obter a cor associada à emoção (color)
  emotion = getEmotion(emotions)
  color = getEmotionColor(emotion)

  increase_amout = 20 #aumentar o tamanho do retângulo para cima, para baixo, para a esquerda e para a direita.
  row1 -= increase_amout
  row2 += increase_amout
  col1 -= increase_amout
  col2 += increase_amout

  rect = mat[row1:row2, col1:col2] #armazena uma submatriz
  center = ( (col2-col1)/2, (row2-row1)/2 )
  maxDistance = min(
    euclidianDistance( center, (col2-col1, center[1]) ),
    euclidianDistance( center, (center[0], row2-row1) ),
  )
  grayIntensity = np.array([ .21, .72, .07 ])
  for i, j in np.ndindex(rect.shape[:-1]):
    pix = rect[i][j]
    gray = (pix * grayIntensity).sum() / 255.
    colored = np.array(color * gray, dtype=np.uint8)
    distance = euclidianDistance( center, (j,i) )
    prcDist = min(1, max(0, distance / maxDistance - 0.2))
    finalColor = pix * prcDist + colored * (1-prcDist)
    rect[i][j] = np.array(finalColor, dtype=np.uint8)

def printEmotionsOnFaces(image_url):
  image_path = image_url.split('/')[-1].split('?')[0]
  urllib.request.urlretrieve(image_url, image_path)
  im = Image.open(image_path)
  mat = np.array(im)
  faces = getFaces(image_url)
  print('faces:', faces)
  for face in faces:
    markEmotionOnFace(face, mat)
  return mat

def getFacesWithEmotions(image_url):
  mat = printEmotionsOnFaces(image_url)
  Image.fromarray(mat).save('emotionfull.png')
  plt.imshow(mat)
  plt.show()



In [ ]:

dir_path = '/content/drive/MyDrive/expressoes faciais dataset'
# Define o diretório onde os arquivos de imagem estão localizados
emotions = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
# Define uma lista de emoções que serão processadas
results = pd.DataFrame(columns=['id', 'real_emotion', 'pred_emotion'])
# Cria um DataFrame vazio para armazenar os resultados das emoções detectadas
num_samples = 100
# Define o número de amostras por emoção que serão processadas
for emotion in emotions: # Itera sobre cada emoção na lista de emoções
  arquivos = os.listdir(f'{dir_path}/{emotion}')
  arquivos_jpg = [arquivo for arquivo in arquivos if arquivo.endswith('.jpg')]
   # Filtra apenas os arquivos com extensão .jpg
  arquivos_jpg = arquivos_jpg[:num_samples]
  # Limita o número de amostras por emoção com base no valor de num_samples
  for arquivo_jpg in arquivos_jpg:   # Itera sobre cada arquivo de imagem .jpg na lista
    img_path = f'{dir_path}/{emotion}/{arquivo_jpg}'  # Cria o caminho completo para o arquivo de imagem
    faces = getFaces(img_path) # Obtém as faces detectadas na imagem usando a função getFaces()
    for face in faces: # Itera sobre cada face detectada
      try:
        pred_emotions = face['faceAttributes']['emotion']  # Obtém as emoções previstas para a face específica
        pred_emotion = getEmotion(pred_emotions) # Obtém a emoção predominante a partir das emoções previstas
        new_result = pd.DataFrame({
            'id': [arquivo_jpg],
            'real_emotion': [emotion],
            'pred_emotion': [pred_emotion]
        })
         # Cria um novo DataFrame com as informações da emoção detectada
        results = pd.concat([results, new_result], ignore_index=True) # Concatena o novo resultado ao DataFrame results
        results.to_csv(f'{dir_path}/results2.csv', index=False) # Salva os resultados em um arquivo CSV chamado 'results2.csv'
      # Caso ocorra uma exceção durante o processamento da face
      except Exception as e:
        with open(f'{dir_path}/erro.txt', 'w') as file:
            traceback.print_exc(file=file)

In [ ]:
from sklearn.metrics import confusion_matrix

df = pd.read_csv('/content/drive/MyDrive/expressoes faciais dataset/results.csv')

# Obter as classes únicas presentes nas colunas
classes = sorted(list(set(df['real_emotion']).union(set(df['pred_emotion']))))

# Gerar a matriz de confusão
confusion = confusion_matrix(df['real_emotion'], df['pred_emotion'], labels=classes)

# Criar um DataFrame a partir da matriz de confusão
confusion_df = pd.DataFrame(confusion, index=classes, columns=classes)

confusion_df

,anger,contempt,disgust,fear,happiness,neutral,sadness,surprise
anger,12,1,0,0,0,8,1,2
contempt,0,0,0,0,0,0,0,0
disgust,0,2,5,0,0,4,6,0
fear,1,0,0,5,0,8,6,5
happiness,0,0,0,0,13,0,0,0
neutral,1,0,0,0,1,21,1,0
sadness,0,0,0,0,0,12,4,0
surprise,0,0,0,0,4,1,0,24


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(df['real_emotion'], df['pred_emotion'])

print("Acurácia:", accuracy)

Acurácia: 0.5675675675675675
